<a href="https://colab.research.google.com/github/lata1207/Master_Project/blob/main/Sentiment_Analysis_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets fsspec==2025.3.2


INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 3.6.0
    Uninstalling datasets-3.6.0:
      Successfully uninsta

In [ ]:
 from datasets import load_dataset

# Load the verified working dataset
dataset = load_dataset("frankdarkluo/dailydialog")

# Preview structure
print(dataset)
print(dataset["train"][0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

valid.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'response'],
        num_rows: 60005
    })
    validation: Dataset({
        features: ['context', 'response'],
        num_rows: 6594
    })
    test: Dataset({
        features: ['context', 'response'],
        num_rows: 6955
    })
})
{'context': 'Waiter ! ', 'response': " I'll be with you in a second . Uh ... Yes , ma'am ? "}


In [ ]:
# Auto-Label Responses with Sentiment
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Load a pre-trained Twitter sentiment model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
classifier = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name)

# Annotate the first 5 responses
for i in range(5):
    text = dataset["train"][i]["response"]
    result = classifier(text)[0]
    print(f"Response: {text}")
    print(f"Predicted Sentiment: {result['label']} (score: {result['score']:.2f})")
    print("---")


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


Response:  I'll be with you in a second . Uh ... Yes , ma'am ? 
Predicted Sentiment: LABEL_1 (score: 0.78)
---
Response:  This is not what I asked for . I'm afraid . 
Predicted Sentiment: LABEL_0 (score: 0.92)
---
Response:  Oh , I'm so sorry . May I ask what you ordered again ? 
Predicted Sentiment: LABEL_0 (score: 0.72)
---
Response:  Yes . What I ordered is roast beef , not roast beef sandwiches . 
Predicted Sentiment: LABEL_1 (score: 0.68)
---
Response:  Mind your own business . 
Predicted Sentiment: LABEL_1 (score: 0.59)
---


In [ ]:
label_map = {
    "LABEL_0": "Negative",
    "LABEL_1": "Neutral",
    "LABEL_2": "Positive"
}

for i in range(5):
    text = dataset["train"][i]["response"]
    result = classifier(text)[0]
    sentiment = label_map[result['label']]
    print(f"Response: {text}")
    print(f"Predicted Sentiment: {sentiment} (score: {result['score']:.2f})")
    print("---")


Response:  I'll be with you in a second . Uh ... Yes , ma'am ? 
Predicted Sentiment: Neutral (score: 0.78)
---
Response:  This is not what I asked for . I'm afraid . 
Predicted Sentiment: Negative (score: 0.92)
---
Response:  Oh , I'm so sorry . May I ask what you ordered again ? 
Predicted Sentiment: Negative (score: 0.72)
---
Response:  Yes . What I ordered is roast beef , not roast beef sandwiches . 
Predicted Sentiment: Neutral (score: 0.68)
---
Response:  Mind your own business . 
Predicted Sentiment: Neutral (score: 0.59)
---


In [ ]:
# Pick a sample from the dataset
sample = dataset["train"][0]

# Extract context and response
context = sample["context"]
response = sample["response"]

# Combine them into one input string
combined_input = f"{context} [SEP] {response}"

print("Combined Input:", combined_input)


Combined Input: Waiter !  [SEP]  I'll be with you in a second . Uh ... Yes , ma'am ? 


In [ ]:
# Load sentiment analysis pipeline
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
classifier = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name)

# Define label mapping
label_map = {
    "LABEL_0": "Negative",
    "LABEL_1": "Neutral",
    "LABEL_2": "Positive"
}

# Test on first 5 samples using context + response
for i in range(5):
    sample = dataset["train"][i]
    context = sample["context"]
    response = sample["response"]

    combined_input = f"{context} [SEP] {response}"

    result = classifier(combined_input)[0]
    sentiment = label_map[result["label"]]

    print(f"Context: {context}")
    print(f"Response: {response}")
    print(f"Combined: {combined_input}")
    print(f"Predicted Sentiment (with context): {sentiment} (score: {result['score']:.2f})")
    print("---")


Device set to use cpu


Context: Waiter ! 
Response:  I'll be with you in a second . Uh ... Yes , ma'am ? 
Combined: Waiter !  [SEP]  I'll be with you in a second . Uh ... Yes , ma'am ? 
Predicted Sentiment (with context): Neutral (score: 0.83)
---
Context:  I'll be with you in a second . Uh ... Yes , ma'am ? 
Response:  This is not what I asked for . I'm afraid . 
Combined:  I'll be with you in a second . Uh ... Yes , ma'am ?  [SEP]  This is not what I asked for . I'm afraid . 
Predicted Sentiment (with context): Negative (score: 0.82)
---
Context:  This is not what I asked for . I'm afraid . 
Response:  Oh , I'm so sorry . May I ask what you ordered again ? 
Combined:  This is not what I asked for . I'm afraid .  [SEP]  Oh , I'm so sorry . May I ask what you ordered again ? 
Predicted Sentiment (with context): Negative (score: 0.92)
---
Context:  Oh , I'm so sorry . May I ask what you ordered again ? 
Response:  Yes . What I ordered is roast beef , not roast beef sandwiches . 
Combined:  Oh , I'm so sorry .

In [ ]:
# Compare performance — with vs without context
contextual_labels = []
non_contextual_labels = []

for i in range(100):  # Use 100 samples for speed
    sample = dataset["train"][i]
    context = sample["context"]
    response = sample["response"]

    # Without context
    no_context_result = classifier(response)[0]
    no_context_sentiment = label_map[no_context_result["label"]]
    non_contextual_labels.append(no_context_sentiment)

    # With context
    combined_input = f"{context} [SEP] {response}"
    context_result = classifier(combined_input)[0]
    context_sentiment = label_map[context_result["label"]]
    contextual_labels.append(context_sentiment)

    if no_context_sentiment != context_sentiment:
        print(f"--- Sample {i} ---")
        print(f"Context: {context}")
        print(f"Response: {response}")
        print(f"Without Context: {no_context_sentiment}")
        print(f"With Context:    {context_sentiment}")
        print()


--- Sample 3 ---
Context:  Oh , I'm so sorry . May I ask what you ordered again ? 
Response:  Yes . What I ordered is roast beef , not roast beef sandwiches . 
Without Context: Neutral
With Context:    Negative

--- Sample 4 ---
Context: Why are you always staring at Melissa ? Do you like her or something ? 
Response:  Mind your own business . 
Without Context: Neutral
With Context:    Negative

--- Sample 6 ---
Context:  A month . Don't forget to return them by the due date . 
Response:  What if I can't finish them by then ? 
Without Context: Negative
With Context:    Neutral

--- Sample 13 ---
Context:  yes , we can look at the computer and computer menu at the same time . And this will help us understand both better . 
Response:  What should I learn first ? 
Without Context: Neutral
With Context:    Positive

--- Sample 17 ---
Context:  Since summer is coming , I think swimming is a good way for you to do . 
Response:  Are you sure ? 
Without Context: Neutral
With Context:    Positi

In [ ]:
# Compute statistics (optional)
# count how often sentiment changes due to context

from collections import Counter

diff_count = sum([c != nc for c, nc in zip(contextual_labels, non_contextual_labels)])
print(f"\nSentiment changed in {diff_count}/100 samples ({diff_count}%) due to context.")



Sentiment changed in 24/100 samples (24%) due to context.


In [ ]:
# Load second sentiment model (fine-tuned BERT on SST-2)
second_model = "distilbert-base-uncased-finetuned-sst-2-english"
second_classifier = pipeline("sentiment-analysis", model=second_model)

# Label mapping for SST-2
label_map_sst = {
    "NEGATIVE": "Negative",
    "POSITIVE": "Positive"
}

# Run on first 5 samples
for i in range(5):
    context = dataset["train"][i]["context"]
    response = dataset["train"][i]["response"]

    combined_input = f"{context} [SEP] {response}"

    result_response_only = second_classifier(response)[0]
    result_combined = second_classifier(combined_input)[0]

    sentiment_response_only = label_map_sst[result_response_only["label"]]
    sentiment_combined = label_map_sst[result_combined["label"]]

    print(f"Sample {i+1}")
    print(f"Context: {context}")
    print(f"Response: {response}")
    print(f"Sentiment (Response Only): {sentiment_response_only} (score: {result_response_only['score']:.2f})")
    print(f"Sentiment (With Context): {sentiment_combined} (score: {result_combined['score']:.2f})")
    print("---")


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Sample 1
Context: Waiter ! 
Response:  I'll be with you in a second . Uh ... Yes , ma'am ? 
Sentiment (Response Only): Positive (score: 1.00)
Sentiment (With Context): Positive (score: 1.00)
---
Sample 2
Context:  I'll be with you in a second . Uh ... Yes , ma'am ? 
Response:  This is not what I asked for . I'm afraid . 
Sentiment (Response Only): Negative (score: 1.00)
Sentiment (With Context): Negative (score: 1.00)
---
Sample 3
Context:  This is not what I asked for . I'm afraid . 
Response:  Oh , I'm so sorry . May I ask what you ordered again ? 
Sentiment (Response Only): Negative (score: 1.00)
Sentiment (With Context): Negative (score: 1.00)
---
Sample 4
Context:  Oh , I'm so sorry . May I ask what you ordered again ? 
Response:  Yes . What I ordered is roast beef , not roast beef sandwiches . 
Sentiment (Response Only): Negative (score: 0.71)
Sentiment (With Context): Negative (score: 1.00)
---
Sample 5
Context: Why are you always staring at Melissa ? Do you like her or somethin